#Load parameters and logger

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
nepochs = 4
batchsize= 8  #8, 16, 32, 64, 128
learning_rate= 5e-5 # 3e-4, 1e-4, 5e-5, 3e-5 
warmup = 0
seed = 0

log_name = '{}_nepochs_{}_batchsize_{}_learning_rate_{}'.format(seed, nepochs, batchsize, learning_rate, warmup)
output = './checkpoints/' + log_name

#Import packages and libraries

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from datasets import load_dataset
from transformers import Trainer
from transformers import TrainingArguments
from datasets import load_metric
import numpy as np

# Load model

In [ ]:
#Load pre-trained phoBERT for finetuning on our dataset
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=8)
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

loading configuration file https://huggingface.co/vinai/phobert-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a596f267f08b7158c7ab6300b1bf98eb6e1b05e6bcb0d7c18a8070364ee3011b.bbe27b2cac909b2279c83792c2d2b6f159f0a95f5d1c1eb66451da1c89a53609
Model config RobertaConfig {
  "_name_or_path": "vinai/phobert-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
 

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Load data

In [ ]:
train_dataset = load_dataset("csv", data_files='/content/drive/MyDrive/nlp_dataset/train_dataset.csv', split='train')
dev_dataset = load_dataset("csv", data_files='/content/drive/MyDrive/nlp_dataset/train_dataset.csv', split="train")

Using custom data configuration default-e335609a57f39d33
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-e335609a57f39d33/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)
Using custom data configuration default-e335609a57f39d33
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-e335609a57f39d33/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=True).shuffle(seed=seed)
dev_dataset = dev_dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-e335609a57f39d33/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-1c80317fa3b1799d.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-e335609a57f39d33/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-bdd640fb06671ad1.arrow


#Train model

In [ ]:
#Hyperparameters
training_args = TrainingArguments(
  output_dir="checkpoint",
  group_by_length=False,
  evaluation_strategy="steps",
  num_train_epochs=nepochs,
  per_device_train_batch_size=batchsize,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  dataloader_num_workers=8, #tune
  learning_rate=learning_rate, #tune
  save_total_limit=15, #tune
  eval_accumulation_steps=2000, #tune
  warmup_ratio=0.1,
  report_to='tensorboard'
)

PyTorch: setting up devices


In [ ]:
metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='weighted')

In [ ]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=dev_dataset, compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Unnamed: 0, text, label_name. If Unnamed: 0, text, label_name are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or s

Step,Training Loss,Validation Loss,F1
500,0.911100,0.431891,0.885309
1000,0.480600,0.333699,0.920615
1500,0.347500,0.247650,0.946468
2000,0.267700,0.172554,0.964409
2500,0.188600,0.160513,0.967319


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Unnamed: 0, text, label_name. If Unnamed: 0, text, label_name are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5145
  Batch size = 8
Saving model checkpoint to checkpoint/checkpoint-500
Configuration saved in checkpoint/checkpoint-500/config.json
Model weights saved in checkpoint/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.


TrainOutput(global_step=2576, training_loss=0.43182177091977614, metrics={'train_runtime': 1521.8531, 'train_samples_per_second': 13.523, 'train_steps_per_second': 1.693, 'total_flos': 2707558612254720.0, 'train_loss': 0.43182177091977614, 'epoch': 4.0})

In [ ]:
model.save_pretrained(output)

Configuration saved in ./checkpoints/3_nepochs_4_batchsize_8_learning_rate_5e-05/config.json
Model weights saved in ./checkpoints/3_nepochs_4_batchsize_8_learning_rate_5e-05/pytorch_model.bin


#Eval

In [ ]:
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
from tqdm import tqdm

In [ ]:
device = 'cuda'

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/nlp_dataset/test_dataset.csv')

predictions = []
labels = []

In [ ]:
for i in tqdm(range(data.shape[0])):
    input_str = data['text'][i]

    input_values = tokenizer(input_str, return_tensors="pt", truncation=True, padding='max_length').input_ids.to(device)
    logits = model(input_values).logits

    prediction = np.argmax(logits.cpu().detach().numpy(), axis=-1)[0]

    labels.append(int(data['label'][i]))
    predictions.append(int(prediction))

100%|██████████| 1287/1287 [00:26<00:00, 49.36it/s]


In [ ]:
target_names = ['cong-nghe',
 'doi-song',
 'du-lich',
 'giai-tri',
 'kinh-doanh-tai-chinh',
 'suc-khoe',
 'the-gioi',
 'the-thao']
print(classification_report(labels, predictions, target_names=target_names, digits=4))

In [ ]:
with open(output + '.txt', 'w') as f:
  f.write(classification_report(labels, predictions, target_names=target_names, digits=4))